<a href="https://colab.research.google.com/github/peiyaow/dl790/blob/master/HW5_Peiyao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
lr = 1e-3
N, num_epochs, H = 1024, 10, 128
# For the moment, we will just train on CPU, so no cuda
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

In [0]:
trainset = torchvision.datasets.CIFAR10(root='.', train=True, transform=transforms.ToTensor(), download = False)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='.', train=False, transform=transforms.ToTensor(), download = False)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False)

In [0]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [52]:
images.view(-1, 3072, 1).shape

torch.Size([64, 3072, 1])

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(3072, H)
        self.dense = nn.Linear(H, 10)

    def forward(self, x):
        x_flat = x.view(-1, 1, 3072)
        out, _ = self.lstm(x_flat)
        return F.log_softmax(self.dense(out)).view(-1, 10)
      
# dl model
model = Net()

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum = 0.5)

In [0]:
def train(model, device, train_loader, optimizer, epoch, train_losses):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())

In [0]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [64]:
train_losses = []
for epoch in range(1, num_epochs + 1):
        train(model, device, trainloader, optimizer, epoch, train_losses)
        test(model, device, testloader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.313770
Train Epoch: 1 [1000/50000 (2%)]	Loss: 2.296269
Train Epoch: 1 [2000/50000 (4%)]	Loss: 2.303172
Train Epoch: 1 [3000/50000 (6%)]	Loss: 2.291117
Train Epoch: 1 [4000/50000 (8%)]	Loss: 2.281386
Train Epoch: 1 [5000/50000 (10%)]	Loss: 2.273923
Train Epoch: 1 [6000/50000 (12%)]	Loss: 2.310280
Train Epoch: 1 [7000/50000 (14%)]	Loss: 2.291764
Train Epoch: 1 [8000/50000 (16%)]	Loss: 2.310677
Train Epoch: 1 [9000/50000 (18%)]	Loss: 2.305898
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.264831
Train Epoch: 1 [11000/50000 (22%)]	Loss: 2.296572
Train Epoch: 1 [12000/50000 (24%)]	Loss: 2.305578
Train Epoch: 1 [13000/50000 (26%)]	Loss: 2.265892
Train Epoch: 1 [14000/50000 (28%)]	Loss: 2.226038
Train Epoch: 1 [15000/50000 (30%)]	Loss: 2.223273
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.219779
Train Epoch: 1 [17000/50000 (34%)]	Loss: 2.233723
Train Epoch: 1 [18000/50000 (36%)]	Loss: 2.218928
Train Epoch: 1 [19000/50000 (38%)]	Loss: 2.196588
Train Epoch: 1 [20